In [1]:
%matplotlib inline
    
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor 

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [3]:
# final df for model building
df1 = pd.read_csv("ML_model_wk40_to_20.csv")
df1.dropna(how='any', inplace=True)
del df1['ILI_weeks']
del df1['Unnamed: 0']
#del df1['week']
df1.head(2)

,Year,week,a_influenza,acute_bronchitis,body_temperature,braun_thermoscan,break_a_fever,bronchitis,chest_cold,cold_and_flu,...,walking_pneumonia,what_to_do_if_you_have_the_flu,Flu_Visit_Count,ILI_Visit_Count,Unspecified,CDC_Unweighted_ILI,ILI_lagwk1,ILI_lagwk2,ILI_lagwk3,ILI_lagwk4
0,2009,40,44,33,81,69,22,40,35,36,...,47,56,0.01338,0.01763,0.03074,5.66087,6.81522,7.61889,7.38836,6.33927
1,2009,41,51,51,77,46,24,43,35,43,...,53,58,0.01620,0.02103,0.03554,6.81522,7.61889,7.38836,6.33927,4.94349


In [4]:
#  combining year and week column 
def combine_year_week(row):
    return int(row["Year"]) * 100 + int(row["week"])

In [5]:
df1["YearWeek"] = df1.apply(combine_year_week, axis=1)

In [6]:
# reset index to YearWeek
df1.index = df1["YearWeek"]

In [7]:
model_df = df1.drop(["Year", "week", "YearWeek"], axis=1)
model_df.head(2)

,a_influenza,acute_bronchitis,body_temperature,braun_thermoscan,break_a_fever,bronchitis,chest_cold,cold_and_flu,cold_or_flu,cold_versus_flu,...,walking_pneumonia,what_to_do_if_you_have_the_flu,Flu_Visit_Count,ILI_Visit_Count,Unspecified,CDC_Unweighted_ILI,ILI_lagwk1,ILI_lagwk2,ILI_lagwk3,ILI_lagwk4
YearWeek,,,,,,,,,,,,,,,,,,,,,
200940,44,33,81,69,22,40,35,36,37,30,...,47,56,0.01338,0.01763,0.03074,5.66087,6.81522,7.61889,7.38836,6.33927
200941,51,51,77,46,24,43,35,43,49,41,...,53,58,0.01620,0.02103,0.03554,6.81522,7.61889,7.38836,6.33927,4.94349


# Independent variables
        # 3 independent variables from athena EHR 
            # [(flu visit counts)/ (total patient visit counts) 
            # (ILI visit counts)/ (total patient visit counts)
            # (unspecified viral or ILI visit counts)/ (total patient visit counts)]
        # CDC historical CDC_Unweighted_ILI values: collected from 2009 to 2016 (week 40 to 20)
        # 129 google search terms related to flu
        
      ======> 3 + 1 + 129
      
# Dependent variables 
        # 4 ILI weeks offset by 1 week

# split df to train dataset, dataset before year2015 and week 40 used at training dataset

In [8]:
train = model_df[model_df.index < 201540]
# train.head()


# defining  targets/labels to  y axis
                y_train = ILI lag week 1

                yy_train = ILI lag week 2

                yyy_train = ILI lag week 3

                yyyy_train = ILI lag week 4

In [9]:
y_train = train["ILI_lagwk1"]
# y_train

In [10]:
yy_train = train["ILI_lagwk2"]

In [11]:
yyy_train = train["ILI_lagwk3"]

In [12]:
yyyy_train =train["ILI_lagwk4"]

In [13]:
X_train = train.drop(["ILI_lagwk1", "ILI_lagwk2", "ILI_lagwk3", "ILI_lagwk4"], axis=1)
# X_train

# split df to train dataset,dataset after year2015 and week 40 used at training dataset

In [14]:
test = model_df[model_df.index >= 201540]
# test.head()

In [15]:
y_test = test["ILI_lagwk1"]

In [16]:
yy_test = test["ILI_lagwk2"]

In [17]:
yyy_test = test["ILI_lagwk3"]

In [18]:
yyyy_test = test["ILI_lagwk4"]

In [19]:
X_test = test.drop(["ILI_lagwk1", "ILI_lagwk2", "ILI_lagwk3", "ILI_lagwk4"], axis=1)

# building a model for y_test = test["ILI_lagwk1"]

In [20]:
rreg1 = RandomForestRegressor(max_depth = 5, n_estimators = 300,  random_state = 0, min_samples_leaf = 8)   

In [21]:
rreg1.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=8, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [22]:
y_pred = rreg1.predict(X_test)

In [23]:
r2_score(y_test, y_pred)

0.8146415710037178

In [24]:
mean_squared_error(y_test, y_pred).round(3)

0.069

In [25]:
mean_absolute_error(y_test, y_pred).round(3)

0.189

In [26]:
print("R-squared for ILI_lagwk1 Train: %.3f" %rreg1.score(X_train, y_train)) 
print("R-squared for ILI_lagwk1 Test : %.3f" %rreg1.score(X_test, y_test)) 

R-squared for ILI_lagwk1 Train: 0.905
R-squared for ILI_lagwk1 Test : 0.815


In [27]:
predictions1 = rreg1.predict(X_test) 

In [28]:
# predict the values of y
predictions = rreg1.predict(X_test)
y_test_unraveled = y_test.values.ravel()
y_test.index

Int64Index([201540, 201541, 201542, 201543, 201544, 201545, 201546, 201547,
            201548, 201549, 201550, 201551, 201552, 201601, 201602, 201603,
            201604, 201605, 201606, 201607, 201608, 201609, 201610, 201611,
            201612, 201613, 201614, 201615, 201616, 201617, 201618, 201619,
            201620],
           dtype='int64', name='YearWeek')

In [29]:
# store prediction and actual to df
pred_df = pd.DataFrame({"Prediction_ILI_lagwk1": predictions1, "Actual": y_test_unraveled}).reset_index(drop=True)
pred_df.index = y_test.index
pred_df.head()

,Prediction_ILI_lagwk1,Actual
YearWeek,,
201540,1.376129,1.33029
201541,1.392518,1.41368
201542,1.483285,1.36882
201543,1.443019,1.48309
201544,1.500482,1.54494


In [30]:
# pred_df.to_csv("Prediction_ILI_lagwk1.csv")
# pred_df.head() 

# # Create the GridSearchCV model

In [31]:
# from sklearn.model_selection import GridSearchCV
# param_grid = {'learning_rate':[ 0.02, 0.01, 0.1, 0.05], 
#             'max_depth':[1, 3, 5 ,7], 
#             'min_samples_leaf':[3, 5, 7 ,9], 
# #             'max_features':[0.1,0.3,1.0],
#             'n_estimators': [300, 500, 1000, 2000]
#              } 

# est = GradientBoostingRegressor()
# gs_cv = GridSearchCV(est, param_grid).fit(X_train, y_train)

# # best hyperparameter setting

# gs_cv.best_est 

# selecting important features  ["ILI_lagwk1"]

In [32]:
# # GBRF predictors variable importance
# store most important variables under importances
importances = rreg1.feature_importances_

In [33]:
rreg1.feature_importances_.sum()

1.0

In [34]:
# # store most important variables/features under importances
importances = rreg1.feature_importances_
importances

array([1.38932970e-05, 4.35173905e-04, 9.20546078e-05, 4.97604542e-04,
       1.58281392e-04, 1.25037250e-04, 2.55789948e-04, 2.31418303e-04,
       4.56881176e-05, 2.51827275e-04, 1.04890533e-04, 4.22792324e-04,
       0.00000000e+00, 2.27948435e-04, 2.59178272e-03, 5.23882026e-05,
       6.05985281e-05, 1.83835639e-04, 2.92054464e-04, 3.36112051e-04,
       2.82347070e-03, 2.57609098e-05, 3.47001963e-04, 1.15083752e-05,
       6.12755761e-05, 2.74897267e-05, 1.72775844e-04, 1.15188993e-03,
       1.78930005e-04, 6.14940260e-04, 1.19063926e-03, 3.03795413e-04,
       1.17425198e-04, 7.19749345e-05, 5.06681213e-06, 7.86955641e-04,
       2.62654798e-04, 0.00000000e+00, 1.05698367e-05, 1.61271843e-05,
       2.95447770e-04, 6.44635625e-05, 9.96466620e-05, 2.17542399e-04,
       3.25309513e-05, 3.93723659e-05, 2.82396075e-05, 2.68546818e-04,
       2.99773576e-03, 4.13578404e-05, 7.35570432e-06, 3.67290487e-05,
       8.52912455e-05, 2.22130732e-06, 1.72542115e-04, 1.31819863e-04,
      

In [35]:
sorted_features = sorted(zip(importances, X_train.columns), reverse=True)
sorted_features

[(0.889018040832374, 'CDC_Unweighted_ILI'),
 (0.028053869812340378, 'type_a_influenza'),
 (0.021163341298380867, 'influenza_type_a'),
 (0.008390317213861933, 'Flu_Visit_Count'),
 (0.003151722741388883, 'ILI_Visit_Count'),
 (0.003065224400364714, 'how_to_treat_flu'),
 (0.0029977357616633744, 'flu_remedy'),
 (0.002823470697018909, 'exposed_to_flu'),
 (0.0028074060797035477, 'get_over_the_flu'),
 (0.00269935251904876, 'sinus'),
 (0.0025917827208937743, 'cure_the_flu'),
 (0.0025490395676404844, 'incubation_period_for_the_flu'),
 (0.002533639792292544, 'treat_the_flu'),
 (0.002441447463452624, 'how_to_treat_the_flu'),
 (0.0020393492769711177, 'low_body'),
 (0.0013745852728133277, 'influenza_treatment'),
 (0.0012624041428047582, 'oscillococcinum'),
 (0.0011906392639627796, 'flu_contagious_period'),
 (0.0011781841753864804, 'tussionex'),
 (0.0011518899255785137, 'flu_care'),
 (0.0009310964040277342, 'walking_pneumonia'),
 (0.0008053404428352178, 'Unspecified'),
 (0.000790363353829208, 'sinus_

In [36]:
# # sort important varibles and sotre them under indices
important_columns = []
for fimportance, name in sorted(zip(importances, X_train.columns), reverse=True):
    if fimportance > 0.00009:
        important_columns.append(name)

In [37]:
important_feature_columns = important_columns
important_feature_columns

['CDC_Unweighted_ILI',
 'type_a_influenza',
 'influenza_type_a',
 'Flu_Visit_Count',
 'ILI_Visit_Count',
 'how_to_treat_flu',
 'flu_remedy',
 'exposed_to_flu',
 'get_over_the_flu',
 'sinus',
 'cure_the_flu',
 'incubation_period_for_the_flu',
 'treat_the_flu',
 'how_to_treat_the_flu',
 'low_body',
 'influenza_treatment',
 'oscillococcinum',
 'flu_contagious_period',
 'tussionex',
 'flu_care',
 'walking_pneumonia',
 'Unspecified',
 'sinus_infections',
 'flu_germs',
 'pneumonia',
 'flu_complications',
 'symptoms_of_pneumonia',
 'braun_thermoscan',
 'treat_flu',
 'over_the_counter_flu_medicine',
 'acute_bronchitis',
 'robitussin',
 'contagious_flu',
 'thermoscan',
 'fever_flu',
 'expectorant',
 'strep_throat',
 'flu_contagious',
 'flu_incubation',
 'early_flu_symptoms',
 'flu_remedies',
 'flu_headache',
 'chest_cold',
 'cold_versus_flu',
 'cold_and_flu',
 'cure_flu',
 'tussin',
 'flu_medicine',
 'remedies_for_the_flu',
 'get_rid_of_the_flu',
 'over_the_counter_flu',
 'tessalon',
 'influenz

In [38]:
important_features74_df = model_df[important_feature_columns]
important_features74_df.head()

,CDC_Unweighted_ILI,type_a_influenza,influenza_type_a,Flu_Visit_Count,ILI_Visit_Count,how_to_treat_flu,flu_remedy,exposed_to_flu,get_over_the_flu,sinus,...,influenza_symptoms,bronchitis,remedies_for_flu,flu_cough,high_fever,cold_vs_flu,signs_of_the_flu,influenza_a_and_b,flu_length,body_temperature
YearWeek,,,,,,,,,,,,,,,,,,,,,
200940,5.66087,78,78,0.01338,0.01763,50,31,59,28,58,...,25,40,31,58,54,48,31,40,28,81
200941,6.81522,77,77,0.01620,0.02103,32,47,78,44,60,...,31,43,37,57,61,52,35,35,44,77
200942,7.61889,94,94,0.02078,0.02626,66,58,90,40,60,...,34,48,55,75,74,59,50,21,35,88
200943,7.38836,100,100,0.02862,0.03500,60,49,99,58,62,...,36,54,51,100,87,82,50,51,51,100
200944,6.33927,77,77,0.02927,0.03515,80,64,99,47,59,...,37,55,55,100,81,73,57,39,57,96


In [39]:
# important_features74_df.to_csv("ML_model_wk40_to_20_with_74_important_features.csv")
# important_features74_df.head()

# building a model for y_test = test["ILI_lagwk2"]

In [40]:
rreg2 = RandomForestRegressor(max_depth = 5, n_estimators = 300,  random_state = 0, min_samples_leaf = 8)   

In [41]:
rreg2.fit(X_train, yy_train) 

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=8, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [42]:
yy_pred = rreg2.predict(X_test)

In [43]:
r2_score(yy_test, yy_pred)

0.5834794077055614

In [44]:
mean_squared_error(yy_test, yy_pred).round(3)

0.153

In [45]:
mean_absolute_error(yy_test, yy_pred).round(3)

0.316

In [46]:
print("R-squared for ILI_lagwk2 Train: %.3f" %rreg2.score(X_train, yy_train)) 
print("R-squared for ILI_lagwk2 Test: %.3f" %rreg2.score(X_test, yy_test)) 

R-squared for ILI_lagwk2 Train: 0.858
R-squared for ILI_lagwk2 Test: 0.583


In [47]:
predictions2 = rreg2.predict(X_test) 

In [48]:
# predict the values of y
yy_test_unraveled = yy_test.values.ravel()
yy_test.index

Int64Index([201540, 201541, 201542, 201543, 201544, 201545, 201546, 201547,
            201548, 201549, 201550, 201551, 201552, 201601, 201602, 201603,
            201604, 201605, 201606, 201607, 201608, 201609, 201610, 201611,
            201612, 201613, 201614, 201615, 201616, 201617, 201618, 201619,
            201620],
           dtype='int64', name='YearWeek')

In [49]:
# store prediction and actual to df
pred_df = pd.DataFrame({"Prediction_ILI_lagwk2": predictions2, "Actual": yy_test_unraveled}).reset_index(drop=True)
pred_df.index = yy_test.index
pred_df.head()

,Prediction_ILI_lagwk2,Actual
YearWeek,,
201540,1.458390,1.41368
201541,1.489996,1.36882
201542,1.559351,1.48309
201543,1.559756,1.54494
201544,1.559101,1.62532


In [50]:
# # Create the GridSearchCV model

# from sklearn.model_selection import GridSearchCV
# param_grid = {'learning_rate':[ 0.02, 0.01, 0.1, 0.05], 
#             'max_depth':[1, 3, 5 ,7], 
#             'min_samples_leaf':[3, 5, 7 ,9], 
# #             'max_features':[0.1,0.3,1.0],
#             'n_estimators': [300, 500, 1000, 2000]
#              } 

# est = GradientBoostingRegressor()
# gs_cv = GridSearchCV(est, param_grid).fit(X_train, y_train)

# # best hyperparameter setting

# gs_cv.best_est 

# building a model for y_test = test["ILI_lagwk3"]

In [51]:
rreg3 = RandomForestRegressor(max_depth = 5, n_estimators = 300,  random_state = 0, min_samples_leaf = 8)   

In [52]:
rreg3.fit(X_train, yyy_train) 

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=8, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [53]:
yyy_pred = rreg3.predict(X_test)

In [54]:
r2_score(yyy_test, yyy_pred)

0.47749076566864534

In [55]:
mean_squared_error(yyy_test, yyy_pred).round(3)

0.198

In [56]:
mean_absolute_error(yyy_test, yyy_pred).round(3)

0.34

In [57]:
print("R-squared for Train: %.3f" %rreg3.score(X_train, yyy_train)) 
print("R-squared for Test: %.3f" %rreg3.score(X_test, yyy_test)) 

R-squared for Train: 0.827
R-squared for Test: 0.477


In [58]:
predictions3 = rreg3.predict(X_test) 

In [59]:
# predict the values of y
predictions = rreg3.predict(X_test)
yyy_test_unraveled = yyy_test.values.ravel()
yyy_test.index

Int64Index([201540, 201541, 201542, 201543, 201544, 201545, 201546, 201547,
            201548, 201549, 201550, 201551, 201552, 201601, 201602, 201603,
            201604, 201605, 201606, 201607, 201608, 201609, 201610, 201611,
            201612, 201613, 201614, 201615, 201616, 201617, 201618, 201619,
            201620],
           dtype='int64', name='YearWeek')

In [60]:
pred_df = pd.DataFrame({"Prediction_ILI_lagwk3": predictions3, "Actual": yyy_test_unraveled}).reset_index(drop=True)
pred_df.index = yyy_test.index
pred_df.head()

,Prediction_ILI_lagwk3,Actual
YearWeek,,
201540,1.566789,1.36882
201541,1.633796,1.48309
201542,1.682643,1.54494
201543,1.697031,1.62532
201544,1.629000,1.91565


# building a model for y_test = test["ILI_lagwk4"]

In [61]:
rreg4 = RandomForestRegressor(max_depth = 5, n_estimators = 300,  random_state = 0, min_samples_leaf = 8)  

In [62]:
rreg4.fit(X_train, yyyy_train) 

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=8, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [63]:
yyyy_pred = rreg4.predict(X_test)

In [64]:
r2_score(yyyy_test, yyyy_pred)

0.34100547625960964

In [65]:
mean_squared_error(yyyy_test, yyyy_pred).round(3)

0.257

In [66]:
mean_absolute_error(yyyy_test, yyyy_pred).round(3)

0.385

In [67]:
print("R-squared for Train: %.3f" %rreg4.score(X_train, yyyy_train)) 
print("R-squared for Test: %.3f" %rreg4.score(X_test, yyyy_test)) 

R-squared for Train: 0.820
R-squared for Test: 0.341


In [68]:
# predict the values of y
predictions4 = rreg4.predict(X_test)
yyyy_test_unraveled = yyyy_test.values.ravel()
yyyy_test.index

Int64Index([201540, 201541, 201542, 201543, 201544, 201545, 201546, 201547,
            201548, 201549, 201550, 201551, 201552, 201601, 201602, 201603,
            201604, 201605, 201606, 201607, 201608, 201609, 201610, 201611,
            201612, 201613, 201614, 201615, 201616, 201617, 201618, 201619,
            201620],
           dtype='int64', name='YearWeek')

In [69]:
# store prediction and actual to df
pred_df = pd.DataFrame({"Prediction_ILI_lagwk1": predictions4, "Actual": yyyy_test_unraveled}).reset_index(drop=True)
pred_df.index = yyyy_test.index
pred_df.head()

,Prediction_ILI_lagwk1,Actual
YearWeek,,
201540,1.601719,1.48309
201541,1.857401,1.54494
201542,1.854575,1.62532
201543,1.960527,1.91565
201544,1.816402,1.74368
